# 클라우드 기반 데이터 시각화 종합 실습

## 실습 내용
- Seaborn을 활용한 통계적 시각화
- Plotly Dash를 활용한 인터랙티브 대시보드
- Folium을 활용한 지도 시각화

## 1. 필요한 라이브러리 임포트

In [ ]:
# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 지도 시각화
import folium
from folium import plugins

# 웹 요청
import requests
from pandas import json_normalize

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 시각화 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 2. Seaborn을 활용한 통계적 시각화

### 2.1 데이터 로드 및 탐색

In [ ]:
# Seaborn 내장 데이터셋 확인
print("사용 가능한 Seaborn 데이터셋:")
print(sns.get_dataset_names()[:10])  # 처음 10개만 표시

In [ ]:
# Tips 데이터셋 로드
tips_df = sns.load_dataset('tips')
print("Tips 데이터셋 정보:")
print(tips_df.info())
print("\n첫 5개 행:")
tips_df.head()

### 2.2 산점도 (Scatter Plot)

In [ ]:
# 총 금액과 팁의 관계 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 기본 산점도
sns.scatterplot(data=tips_df, x='total_bill', y='tip', ax=axes[0])
axes[0].set_title('총 금액 vs 팁', fontsize=14)

# 카테고리별 산점도
sns.scatterplot(data=tips_df, x='total_bill', y='tip', 
                hue='sex', size='size', style='time', ax=axes[1])
axes[1].set_title('카테고리별 총 금액 vs 팁', fontsize=14)

plt.tight_layout()
plt.show()

### 2.3 회귀 플롯 (Regression Plot)

In [ ]:
# 회귀선과 신뢰구간 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 전체 데이터 회귀선
sns.regplot(data=tips_df, x='total_bill', y='tip', ax=axes[0])
axes[0].set_title('회귀선과 95% 신뢰구간', fontsize=14)

# 흡연 여부별 회귀선 비교
for smoker in ['Yes', 'No']:
    subset = tips_df[tips_df['smoker'] == smoker]
    sns.regplot(data=subset, x='total_bill', y='tip', 
                label=f'Smoker: {smoker}', ax=axes[1])
axes[1].set_title('흡연 여부별 회귀선 비교', fontsize=14)
axes[1].legend()

plt.tight_layout()
plt.show()

### 2.4 박스플롯과 바이올린플롯

In [ ]:
# 요일별 팁 분포 비교
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 박스플롯
weekday_order = ['Thur', 'Fri', 'Sat', 'Sun']
sns.boxplot(data=tips_df, x='day', y='tip', order=weekday_order, ax=axes[0, 0])
axes[0, 0].set_title('요일별 팁 분포 (박스플롯)', fontsize=14)

# 바이올린플롯
sns.violinplot(data=tips_df, x='day', y='tip', order=weekday_order, ax=axes[0, 1])
axes[0, 1].set_title('요일별 팁 분포 (바이올린플롯)', fontsize=14)

# 박스플롯 + 스트립플롯
sns.boxplot(data=tips_df, x='day', y='tip', order=weekday_order, ax=axes[1, 0])
sns.stripplot(data=tips_df, x='day', y='tip', order=weekday_order, 
              color='black', alpha=0.3, ax=axes[1, 0])
axes[1, 0].set_title('박스플롯 + 실제 데이터 포인트', fontsize=14)

# 시간대별 요일별 팁 분포
sns.boxplot(data=tips_df, x='day', y='tip', hue='time', 
            order=weekday_order, ax=axes[1, 1])
axes[1, 1].set_title('시간대별 요일별 팁 분포', fontsize=14)

plt.tight_layout()
plt.show()

### 2.5 히트맵 (Heatmap)

In [ ]:
# 피벗 테이블 생성 및 히트맵 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 요일과 파티 크기별 평균 팁
pivot_tips = tips_df.pivot_table(values='tip', index='day', columns='size', aggfunc='mean')
pivot_tips = pivot_tips.fillna(0)
pivot_tips = pivot_tips.reindex(weekday_order)

sns.heatmap(pivot_tips, annot=True, fmt='.2f', cmap='YlOrRd', ax=axes[0])
axes[0].set_title('요일과 파티 크기별 평균 팁', fontsize=14)

# 상관관계 행렬
corr_matrix = tips_df[['total_bill', 'tip', 'size']].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, ax=axes[1])
axes[1].set_title('변수 간 상관관계', fontsize=14)

plt.tight_layout()
plt.show()

### 2.6 FacetGrid를 활용한 다중 플롯

In [ ]:
# FacetGrid로 다차원 데이터 시각화
g = sns.FacetGrid(tips_df, col='time', row='smoker', 
                  hue='sex', height=4, aspect=1.2)
g.map(sns.scatterplot, 'total_bill', 'tip', alpha=0.7)
g.add_legend()
g.fig.suptitle('시간대와 흡연 여부별 총 금액 vs 팁', fontsize=16, y=1.02)
plt.show()

## 3. Plotly를 활용한 인터랙티브 시각화

### 3.1 인터랙티브 산점도

In [ ]:
# Plotly Express를 활용한 인터랙티브 산점도
fig = px.scatter(tips_df, x='total_bill', y='tip',
                 color='smoker', size='size', 
                 hover_data=['day', 'time', 'sex'],
                 title='인터랙티브 팁 데이터 분석',
                 labels={'total_bill': '총 금액', 'tip': '팁'},
                 template='plotly_white')

fig.update_layout(width=800, height=500)
fig.show()

### 3.2 3D 산점도

In [ ]:
# 3D 산점도로 다차원 관계 시각화
fig = px.scatter_3d(tips_df, x='total_bill', y='tip', z='size',
                    color='time', symbol='smoker',
                    title='3D 팁 데이터 분석',
                    labels={'total_bill': '총 금액', 'tip': '팁', 'size': '파티 크기'})

fig.update_layout(width=800, height=600)
fig.show()

### 3.3 애니메이션 차트

In [ ]:
# 요일별 애니메이션 산점도
# 데이터 준비: 요일 순서 정의
tips_df['day_cat'] = pd.Categorical(tips_df['day'], 
                                     categories=['Thur', 'Fri', 'Sat', 'Sun'], 
                                     ordered=True)
tips_sorted = tips_df.sort_values('day_cat')

fig = px.scatter(tips_sorted, x='total_bill', y='tip',
                 animation_frame='day', animation_group='day',
                 color='time', size='size',
                 hover_name='day', hover_data=['sex', 'smoker'],
                 title='요일별 팁 패턴 애니메이션',
                 range_x=[0, 60], range_y=[0, 12])

fig.update_layout(width=800, height=500)
fig.show()

### 3.4 Dash 대시보드 예제 코드

In [ ]:
# Dash 대시보드 예제 코드 (별도 파일로 실행 필요)
dash_code = '''
# dash_app.py
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
import pandas as pd

# Dash 앱 초기화
app = Dash(__name__)

# 데이터 로드
df = px.data.tips()

# 레이아웃 정의
app.layout = html.Div([
    html.H1('팁 데이터 인터랙티브 대시보드', 
            style={'text-align': 'center', 'margin-bottom': '30px'}),
    
    html.Div([
        html.Div([
            html.Label('요일 선택:'),
            dcc.Dropdown(
                id='day-dropdown',
                options=[{'label': 'All', 'value': 'All'}] + 
                        [{'label': day, 'value': day} for day in df['day'].unique()],
                value='All',
                style={'width': '200px'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label('시간대 선택:'),
            dcc.RadioItems(
                id='time-radio',
                options=[{'label': 'All', 'value': 'All'}] + 
                        [{'label': time, 'value': time} for time in df['time'].unique()],
                value='All',
                inline=True
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ], style={'margin-bottom': '30px'}),
    
    html.Div([
        dcc.Graph(id='scatter-plot', style={'width': '49%', 'display': 'inline-block'}),
        dcc.Graph(id='box-plot', style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ]),
    
    html.Div([
        dcc.Graph(id='histogram', style={'width': '100%'})
    ])
])

# 콜백 함수 정의
@callback(
    [Output('scatter-plot', 'figure'),
     Output('box-plot', 'figure'),
     Output('histogram', 'figure')],
    [Input('day-dropdown', 'value'),
     Input('time-radio', 'value')]
)
def update_graphs(selected_day, selected_time):
    # 데이터 필터링
    filtered_df = df.copy()
    if selected_day != 'All':
        filtered_df = filtered_df[filtered_df['day'] == selected_day]
    if selected_time != 'All':
        filtered_df = filtered_df[filtered_df['time'] == selected_time]
    
    # 산점도
    scatter_fig = px.scatter(filtered_df, x='total_bill', y='tip',
                            color='smoker', size='size',
                            title='총 금액 vs 팁',
                            template='plotly_white')
    
    # 박스플롯
    box_fig = px.box(filtered_df, x='day', y='tip',
                     color='time',
                     title='요일별 팁 분포',
                     template='plotly_white')
    
    # 히스토그램
    hist_fig = px.histogram(filtered_df, x='tip',
                            color='time',
                            title='팁 금액 분포',
                            nbins=20,
                            template='plotly_white')
    
    return scatter_fig, box_fig, hist_fig

if __name__ == '__main__':
    app.run_server(debug=True)
'''

print("Dash 대시보드 예제 코드:")
print("="*50)
print(dash_code)
print("="*50)
print("\n위 코드를 'dash_app.py' 파일로 저장 후 실행:")
print("python dash_app.py")

## 4. Folium을 활용한 지도 시각화

### 4.1 기본 지도 생성

In [ ]:
# 서울 시청 좌표로 기본 지도 생성
seoul_loc = [37.5662952, 126.9779451]

# 기본 지도 생성
base_map = folium.Map(
    location=seoul_loc,
    zoom_start=12,
    tiles='OpenStreetMap'
)

# 마커 추가
folium.Marker(
    location=seoul_loc,
    popup='서울시청',
    tooltip='클릭하여 정보 보기',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(base_map)

# 원형 마커 추가
folium.CircleMarker(
    location=[37.5665, 126.9780],
    radius=50,
    popup='서울광장',
    color='blue',
    fill=True,
    fillColor='lightblue'
).add_to(base_map)

base_map

### 4.2 스타벅스 매장 위치 시각화

In [ ]:
# 스타벅스 매장 데이터 수집 함수
def get_starbucks_data():
    """스타벅스 매장 데이터 수집"""
    url = 'https://www.starbucks.co.kr/store/getStore.do?r=8K3XF06R9Q'
    
    payload = {
        'ins_lat': 37.563398,
        'ins_lng': 126.9863309,
        'p_sido_cd': '01',
        'p_gugun_cd': '',
        'in_biz_cd': '',
        'iend': 100,  # 100개 매장만 가져오기
        'set_date': ''
    }
    
    try:
        response = requests.post(url, data=payload)
        starbucks_data = response.json()
        
        # 데이터프레임으로 변환
        df = json_normalize(starbucks_data, 'list')
        
        # 필요한 컬럼만 선택
        columns = ['s_name', 'doro_address', 'lat', 'lot']
        df_filtered = df[columns].copy()
        
        # 데이터 타입 변환
        df_filtered['lat'] = pd.to_numeric(df_filtered['lat'], errors='coerce')
        df_filtered['lot'] = pd.to_numeric(df_filtered['lot'], errors='coerce')
        
        return df_filtered.dropna()
    except Exception as e:
        print(f"데이터 수집 오류: {e}")
        # 샘플 데이터 반환
        return pd.DataFrame({
            's_name': ['역삼점', '강남역점', '신논현점'],
            'doro_address': ['서울특별시 강남구 역삼동', '서울특별시 강남구 강남대로', '서울특별시 강남구 논현동'],
            'lat': [37.501087, 37.497942, 37.509892],
            'lot': [127.043069, 127.027617, 127.031306]
        })

# 데이터 수집
starbucks_df = get_starbucks_data()
print(f"수집된 매장 수: {len(starbucks_df)}")
starbucks_df.head()

In [ ]:
# 스타벅스 매장 지도 시각화
starbucks_map = folium.Map(
    location=[37.5665, 126.9780],
    zoom_start=11,
    tiles='CartoDB positron'
)

# 매장별 마커 추가
for idx, row in starbucks_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lot']],
        popup=folium.Popup(f"<b>{row['s_name']}</b><br>{row['doro_address']}", max_width=200),
        tooltip=row['s_name'],
        icon=folium.Icon(color='green', icon='coffee', prefix='fa')
    ).add_to(starbucks_map)

# 매장 클러스터 추가
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(starbucks_map)

for idx, row in starbucks_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lot']],
        popup=row['s_name']
    ).add_to(marker_cluster)

starbucks_map

### 4.3 히트맵 시각화

In [ ]:
# 히트맵을 위한 데이터 준비
from folium.plugins import HeatMap

# 히트맵 지도 생성
heat_map = folium.Map(
    location=[37.5665, 126.9780],
    zoom_start=11,
    tiles='CartoDB dark_matter'
)

# 히트맵 데이터 준비 (위도, 경도, 가중치)
heat_data = [[row['lat'], row['lot'], 1] for idx, row in starbucks_df.iterrows()]

# 히트맵 추가
HeatMap(heat_data, radius=15).add_to(heat_map)

# 타이틀 추가
title_html = '''
             <h3 align="center" style="font-size:20px"><b>서울 스타벅스 매장 밀집도</b></h3>
             '''
heat_map.get_root().html.add_child(folium.Element(title_html))

heat_map

### 4.4 다양한 지도 스타일

In [ ]:
# 다양한 타일 스타일 비교
tiles_list = ['OpenStreetMap', 'CartoDB positron', 'CartoDB dark_matter', 'Stamen Terrain']

# 2x2 그리드로 지도 생성
from IPython.display import HTML

html_maps = []
for tile in tiles_list:
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=10, tiles=tile, width='50%', height='300px')
    folium.Marker(location=[37.5665, 126.9780], popup=f'{tile} Style').add_to(m)
    html_maps.append(m._repr_html_())

# HTML로 결합하여 표시
combined_html = f'''
<div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
    <div>{html_maps[0]}</div>
    <div>{html_maps[1]}</div>
    <div>{html_maps[2]}</div>
    <div>{html_maps[3]}</div>
</div>
'''

HTML(combined_html)

## 5. 종합 실습: 대화형 대시보드 구성

In [ ]:
# Plotly Subplots를 활용한 대시보드
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 서브플롯 생성
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('산점도', '박스플롯', '히스토그램', '파이차트'),
    specs=[[{'type': 'scatter'}, {'type': 'box'}],
           [{'type': 'histogram'}, {'type': 'pie'}]]
)

# 1. 산점도
fig.add_trace(
    go.Scatter(x=tips_df['total_bill'], y=tips_df['tip'],
               mode='markers',
               marker=dict(color=tips_df['size'], colorscale='Viridis', showscale=True),
               text=tips_df['day'],
               name='Tips'),
    row=1, col=1
)

# 2. 박스플롯
for day in tips_df['day'].unique():
    day_data = tips_df[tips_df['day'] == day]['tip']
    fig.add_trace(
        go.Box(y=day_data, name=day),
        row=1, col=2
    )

# 3. 히스토그램
fig.add_trace(
    go.Histogram(x=tips_df['tip'], nbinsx=20, name='Tip Distribution'),
    row=2, col=1
)

# 4. 파이차트
day_counts = tips_df['day'].value_counts()
fig.add_trace(
    go.Pie(labels=day_counts.index, values=day_counts.values),
    row=2, col=2
)

# 레이아웃 업데이트
fig.update_layout(
    title_text="팁 데이터 종합 대시보드",
    showlegend=False,
    height=700,
    template='plotly_white'
)

# 축 레이블 추가
fig.update_xaxes(title_text="총 금액", row=1, col=1)
fig.update_yaxes(title_text="팁", row=1, col=1)
fig.update_yaxes(title_text="팁 금액", row=1, col=2)
fig.update_xaxes(title_text="팁 금액", row=2, col=1)
fig.update_yaxes(title_text="빈도", row=2, col=1)

fig.show()

## 6. 실습 요약 및 정리

### 학습한 내용 정리

#### 1. **Seaborn**
- 통계적 데이터 시각화에 특화
- 다양한 플롯 유형: 산점도, 회귀플롯, 박스플롯, 바이올린플롯, 히트맵
- FacetGrid를 통한 다차원 데이터 시각화
- Pandas 데이터프레임과의 뛰어난 통합성

#### 2. **Plotly**
- 인터랙티브 시각화 구현
- 3D 시각화 및 애니메이션 지원
- Dash를 통한 웹 대시보드 구축
- 다양한 차트 유형과 커스터마이징 옵션

#### 3. **Folium**
- 지도 기반 데이터 시각화
- 마커, 팝업, 툴팁 추가
- 히트맵과 클러스터링
- 다양한 지도 타일 스타일

### 실무 활용 팁

1. **데이터 탐색 단계**: Seaborn으로 빠른 시각화와 통계 분석
2. **프레젠테이션**: Plotly로 인터랙티브 차트 생성
3. **웹 배포**: Dash나 Streamlit으로 대시보드 구축
4. **지리 데이터**: Folium으로 위치 기반 분석

### 다음 단계

- Streamlit을 활용한 웹 앱 배포
- 실제 데이터셋을 활용한 프로젝트
- 머신러닝 결과 시각화
- 실시간 데이터 모니터링 대시보드 구축